# Creating, Sending and Receiving Packets

In [1]:
from scapy.all import *

In [2]:
conf.verb=1
conf.color_theme = RastaTheme()

## Stacking Layers

- the `/` operator is used as a composition operator between two layers
- the lower layer can have one or more of its defaults fields overloaded according to the upper layer
    - you still can give the value you want
    - a string can be used as a raw layer
    
<img src="http://scapy.readthedocs.io/en/latest/_images/fieldsmanagement.png" />

In [3]:
IP()

<IP  |>

In [4]:
IP()/TCP()

<IP  frag=0 proto=tcp |<TCP  |>>

In [5]:
Ether()/IP()/TCP()

<Ether  type=IPv4 |<IP  frag=0 proto=tcp |<TCP  |>>>

In [6]:
IP()/TCP()/"GET / HTTP/1.0\r\n\r\n"

<IP  frag=0 proto=tcp |<TCP  |<Raw  load='GET / HTTP/1.0\r\n\r\n' |>>>

In [7]:
Ether()/IP()/IP()/UDP()

<Ether  type=IPv4 |<IP  frag=0 proto=ipencap |<IP  frag=0 proto=udp |<UDP  |>>>>

In [8]:
IP(proto=55)/TCP(dport=-22)

<IP  frag=0 proto=55 |<TCP  dport=-22 |>>

## Build ARP packets
- build ARP packet which has two layers - Ether and ARP

In [9]:
ls(Ether)

dst        : DestMACField                        = ('None')
src        : SourceMACField                      = ('None')
type       : XShortEnumField                     = ('36864')


In [10]:
ls(ARP)

hwtype     : XShortEnumField                     = ('1')
ptype      : XShortEnumField                     = ('2048')
hwlen      : FieldLenField                       = ('None')
plen       : FieldLenField                       = ('None')
op         : ShortEnumField                      = ('1')
hwsrc      : MultipleTypeField (SourceMACField, StrFixedLenField) = ('None')
psrc       : MultipleTypeField (SourceIPField, SourceIP6Field, StrFixedLenField) = ('None')
hwdst      : MultipleTypeField (MACField, StrFixedLenField) = ('None')
pdst       : MultipleTypeField (IPField, IP6Field, StrFixedLenField) = ('None')


In [11]:
arpPkt = Ether()/ARP()

In [12]:
arpPkt.show()

###[ Ethernet ]### 
  dst       = 00:50:56:f6:54:9a
  src       = 00:0c:29:8a:20:62
  type      = ARP
###[ ARP ]### 
     hwtype    = Ethernet (10Mb)
     ptype     = IPv4
     hwlen     = None
     plen      = None
     op        = who-has
     hwsrc     = 00:0c:29:8a:20:62
     psrc      = 192.168.47.130
     hwdst     = 00:00:00:00:00:00
     pdst      = 0.0.0.0



In [13]:
# Change Ethernet protocol's destination field to broadcase MAC address
arpPkt['Ethernet'].dst= 'FF:FF:FF:FF:FF:FF'

In [14]:
arpPkt.show()

###[ Ethernet ]### 
  dst       = FF:FF:FF:FF:FF:FF
  src       = 00:0c:29:8a:20:62
  type      = ARP
###[ ARP ]### 
     hwtype    = Ethernet (10Mb)
     ptype     = IPv4
     hwlen     = None
     plen      = None
     op        = who-has
     hwsrc     = 00:0c:29:8a:20:62
     psrc      = 192.168.47.130
     hwdst     = 00:00:00:00:00:00
     pdst      = 0.0.0.0



In [15]:
# change ARP protocol's hwdst to broadcast MAC address
#arpPkt['Ethernet']['ARP'].hwdst = 'FF:FF:FF:FF:FF:FF'
arpPkt['ARP'].hwdst = 'FF:FF:FF:FF:FF:FF'

In [16]:
arpPkt.show()

###[ Ethernet ]### 
  dst       = FF:FF:FF:FF:FF:FF
  src       = 00:0c:29:8a:20:62
  type      = ARP
###[ ARP ]### 
     hwtype    = Ethernet (10Mb)
     ptype     = IPv4
     hwlen     = None
     plen      = None
     op        = who-has
     hwsrc     = 00:0c:29:8a:20:62
     psrc      = 192.168.47.130
     hwdst     = FF:FF:FF:FF:FF:FF
     pdst      = 0.0.0.0



In [27]:
# change dest ip - use Metaploitable IP
arpPkt['ARP'].pdst = '192.168.47.129'

In [28]:
arpPkt.show()

###[ Ethernet ]### 
  dst       = FF:FF:FF:FF:FF:FF
  src       = 00:0c:29:8a:20:62
  type      = ARP
###[ ARP ]### 
     hwtype    = Ethernet (10Mb)
     ptype     = IPv4
     hwlen     = None
     plen      = None
     op        = who-has
     hwsrc     = 00:0c:29:8a:20:62
     psrc      = 192.168.47.130
     hwdst     = FF:FF:FF:FF:FF:FF
     pdst      = 192.168.47.129



## Capture packet using wireshark

- ARP packet is ready to be sent
- Makesure Ethernet source is selected
- Open Wireshark and start capturing traffic
- use arp filter to display only ARP packets

# Send packets - one way
- sendp() - works at layer two; can send ARP, Ethernet, etc.
- send() - send packets at layer 3 (IP, TCP, HTTP, etc.); will handle routing and layer 2
    - if return_packets=True is passed as parameter, these functions will also return received packes

In [29]:
help(send)

Help on function send in module scapy.sendrecv:

send(x, iface=None, **kargs)
    Send packets at layer 3
    
    :param x: the packets
    :param inter: time (in s) between two packets (default 0)
    :param loop: send packet indefinitely (default 0)
    :param count: number of packets to send (default None=1)
    :param verbose: verbose mode (default None=conf.verb)
    :param realtime: check that a packet was sent before sending the next one
    :param return_packets: return the sent packets
    :param socket: the socket to use (default is conf.L3socket(kargs))
    :param iface: the interface to send the packets on
    :param monitor: (not on linux) send in monitor mode
    :returns: None



In [30]:
help(sendp)

Help on function sendp in module scapy.sendrecv:

sendp(x, iface=None, iface_hint=None, socket=None, **kargs)
    Send packets at layer 2
    
    :param x: the packets
    :param inter: time (in s) between two packets (default 0)
    :param loop: send packet indefinitely (default 0)
    :param count: number of packets to send (default None=1)
    :param verbose: verbose mode (default None=conf.verb)
    :param realtime: check that a packet was sent before sending the next one
    :param return_packets: return the sent packets
    :param socket: the socket to use (default is conf.L3socket(kargs))
    :param iface: the interface to send the packets on
    :param monitor: (not on linux) send in monitor mode
    :returns: None



In [31]:
# send the arpPkt - use sendp() because ARP is layer 2 packet
sendp(arpPkt)


Sent 1 packets..


In [32]:
response = srp1(arpPkt)

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


In [33]:
response.show()

###[ Ethernet ]### 
  dst       = 00:0c:29:8a:20:62
  src       = 00:0c:29:b0:50:a7
  type      = ARP
###[ ARP ]### 
     hwtype    = Ethernet (10Mb)
     ptype     = IPv4
     hwlen     = 6
     plen      = 4
     op        = is-at
     hwsrc     = 00:0c:29:b0:50:a7
     psrc      = 192.168.47.129
     hwdst     = 00:0c:29:8a:20:62
     pdst      = 192.168.47.130
###[ Padding ]### 
        load      = '\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'



In [34]:
# doesn't show IP just with ARP
! arp -n

Address                  HWtype  HWaddress           Flags Mask            Iface
192.168.47.2             ether   00:50:56:f6:54:9a   C                     eth0


In [35]:
# one liner
sendp(Ether(dst="ff:ff:ff:ff:ff:ff",src="00:11:22:aa:bb:cc")/ARP(hwsrc="00:11:22:aa:bb:cc",pdst="192.168.231.2"))

.
Sent 1 packets.


## Sending HTTP request

In [64]:
# Metaploitable runs http server on port 80
# Most real-world websites use 443
httpPkt = IP(dst="192.168.47.129")/TCP(dport=80)/"GET / HTTP/1.0\n\n"

In [65]:
bytes(httpPkt)

b'E\x00\x008\x00\x01\x00\x00@\x06\x9ak\xc0\xa8/\x82\xc0\xa8/\x81\x00\x14\x00P\x00\x00\x00\x00\x00\x00\x00\x00P\x02 \x00\xe0\x84\x00\x00GET / HTTP/1.0\n\n'

In [66]:
hexdump(httpPkt)

0000  45 00 00 38 00 01 00 00 40 06 9A 6B C0 A8 2F 82  E..8....@..k../.
0010  C0 A8 2F 81 00 14 00 50 00 00 00 00 00 00 00 00  ../....P........
0020  50 02 20 00 E0 84 00 00 47 45 54 20 2F 20 48 54  P. .....GET / HT
0030  54 50 2F 31 2E 30 0A 0A                          TP/1.0..


In [67]:
httpPkt.show()

###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = None
  src       = 192.168.47.130
  dst       = 192.168.47.129
  \options   \
###[ TCP ]### 
     sport     = ftp_data
     dport     = http
     seq       = 0
     ack       = 0
     dataofs   = None
     reserved  = 0
     flags     = S
     window    = 8192
     chksum    = None
     urgptr    = 0
     options   = ''
###[ Raw ]### 
        load      = 'GET / HTTP/1.0\n\n'



## Send and Receive Packets
- Send and receive functions are the heart of Scapy
- sr() - send L3 packets and receive answers; returns answered and unanswered packets
- sr1() - returns only one packet that answered the sent packet
- sr() and sr1() functions can only send layer 3 L3 packets (IP, HTTP, etc.)
- srp() - send and receive L2 packets (ARP, Ehternet, 802.3, etc)
- srp1() - send and receive one layer 2 packet

In [68]:
response = sr(httpPkt)

Begin emission:
Finished sending 1 packets.

Received 2 packets, got 1 answers, remaining 0 packets


In [69]:
# get the answer tuple
ans = response[0]

In [53]:
help(ans)

Help on SndRcvList in module scapy.plist object:

class SndRcvList(_PacketList, scapy.base_classes.BasePacketList, scapy.base_classes._CanvasDumpExtended)
 |  SndRcvList(res=None, name='Results', stats=None)
 |  
 |  Method resolution order:
 |      SndRcvList
 |      _PacketList
 |      scapy.base_classes.BasePacketList
 |      scapy.base_classes.Gen
 |      typing.Generic
 |      scapy.base_classes._CanvasDumpExtended
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, res=None, name='Results', stats=None)
 |      create a packet list from a list of packets
 |      res: the list of packets
 |      stats: a list of classes that will appear in the stats (defaults to [TCP,UDP,ICMP])
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  -----

In [70]:
ans.show()

0000 IP / TCP 192.168.47.130:ftp_data > 192.168.47.129:http S / Raw ==> IP / TCP 192.168.47.129:http > 192.168.47.130:ftp_data SA / Padding


In [77]:
ans[0]

QueryAnswer(query=<IP  frag=0 proto=tcp dst=192.168.47.129 |<TCP  dport=http |<Raw  load='GET / HTTP/1.0\n\n' |>>>, answer=<IP  version=4 ihl=5 tos=0x0 len=44 id=0 flags=DF frag=0 ttl=64 proto=tcp chksum=0x5a78 src=192.168.47.129 dst=192.168.47.130 |<TCP  sport=http dport=ftp_data seq=3053942448 ack=1 dataofs=6 reserved=0 flags=SA window=5840 chksum=0x73d5 urgptr=0 options=[('MSS', 1460)] |<Padding  load='\x00\x00' |>>>)

## TCP/IP full three-way handshake

### send SYN packet

In [89]:
# Create syn packet
host = '192.168.47.129'
syn = IP(dst=f'{host}')/TCP(dport=80, flags='S')

In [90]:
syn.show()

###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = None
  src       = 192.168.47.130
  dst       = 192.168.47.129
  \options   \
###[ TCP ]### 
     sport     = ftp_data
     dport     = http
     seq       = 0
     ack       = 0
     dataofs   = None
     reserved  = 0
     flags     = S
     window    = 8192
     chksum    = None
     urgptr    = 0
     options   = ''



In [91]:
# Receive SYN-ACK packet
syn_ack = sr1(syn)

Begin emission:
Finished sending 1 packets.

Received 2 packets, got 1 answers, remaining 0 packets


In [92]:
syn_ack.show()

###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 44
  id        = 0
  flags     = DF
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = 0x5a78
  src       = 192.168.47.129
  dst       = 192.168.47.130
  \options   \
###[ TCP ]### 
     sport     = http
     dport     = ftp_data
     seq       = 1779287970
     ack       = 1
     dataofs   = 6
     reserved  = 0
     flags     = SA
     window    = 5840
     chksum    = 0x6edd
     urgptr    = 0
     options   = [('MSS', 1460)]
###[ Padding ]### 
        load      = '\x00\x00'



In [93]:
# Send Get request
getStr = f'GET / HTTP1.1\r\nHost:{host}\r\n\r\n'
request = IP(dst=f'{host}') / TCP(dport=443, sport=syn_ack[TCP].dport, 
                                        seq=syn_ack[TCP].ack, ack=syn_ack[TCP].seq+1, 
                                        flags='A') / getStr

In [94]:
request.show()

###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = None
  src       = 192.168.47.130
  dst       = 192.168.47.129
  \options   \
###[ TCP ]### 
     sport     = ftp_data
     dport     = https
     seq       = 1
     ack       = 1779287971
     dataofs   = None
     reserved  = 0
     flags     = A
     window    = 8192
     chksum    = None
     urgptr    = 0
     options   = ''
###[ Raw ]### 
        load      = 'GET / HTTP1.1\r\nHost:192.168.47.129\r\n\r\n'



In [95]:
# send and get response
reply = sr1(request)

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


In [96]:
reply.show()

###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 40
  id        = 0
  flags     = DF
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = 0x5a7c
  src       = 192.168.47.129
  dst       = 192.168.47.130
  \options   \
###[ TCP ]### 
     sport     = https
     dport     = ftp_data
     seq       = 1779287971
     ack       = 0
     dataofs   = 5
     reserved  = 0
     flags     = R
     window    = 0
     chksum    = 0x9c0c
     urgptr    = 0
     options   = ''
###[ Padding ]### 
        load      = '\x00\x00\x00\x00\x00\x00'



In [52]:
reply.summary()

'IP / TCP 93.184.216.34:http > 192.168.195.161:ftp_data R / Padding'

In [53]:
tcpip = IP(dst="example.com") / TCP(dport=80) # tcp packet needs dest port
reply = sr(tcpip)
reply

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


(<Results: TCP:1 UDP:0 ICMP:0 Other:0>,
 <Unanswered: TCP:0 UDP:0 ICMP:0 Other:0>)

In [54]:
ans, unans = reply 

In [55]:
ans.summary()

IP / TCP 192.168.195.161:ftp_data > 93.184.216.34:http S ==> IP / TCP 93.184.216.34:http > 192.168.195.161:ftp_data SA / Padding


In [56]:
unans

<Unanswered: TCP:0 UDP:0 ICMP:0 Other:0>

## TCP Syn Flooding
-  Not quite working with Scapy but should world with netwox
- form of DoS attack where attacker doesn't complete 3-way handshake by not sending ACK packet after receiving SYN-ACK from server
- attacker typically floods the victim's half-opened connections
- when queue is full, victim cannot take any more new connection
- See Taks 1 here: http://www.cis.syr.edu/~wedu/seed/Labs_16.04/Networking/TCP_Attacks/TCP_Attacks.pdf
- queue size can be viwed by using the following command

In [58]:
# on a server VM, run the following command
# Run SEEDLab VM which has Apache server running
! sysctl -q net.ipv4.tcp_max_syn_backlog

net.ipv4.tcp_max_syn_backlog = 512


In [ ]:
# set max backlog/queue size to 1
! sudo sysctl -w net.ipv4.tcp_max_syn_backlog=2

In [59]:
# Turn of SYN cookie countermeasure
! sudo sysctl -a | grep cookie #(Display the SYN cookie flag)

net.ipv4.tcp_syncookies = 1
net.netfilter.nf_conntrack_sctp_timeout_cookie_echoed = 3
net.netfilter.nf_conntrack_sctp_timeout_cookie_wait = 3


In [ ]:
! sysctl -w net.ipv4.tcp_syncookies=0 #(turn off SYN cookie)

In [ ]:
# the following command shows all the halfopen connections
! netstat -na | grep SYN-RECV

In [101]:
# send SYN PACKET To the server and check for SYN-RECV status
#while True:
pkt = IP(dst=f"{host}")/TCP(dport=80)
reply = sr1(pkt)

Begin emission:
Finished sending 1 packets.

Received 2 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


...

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 15 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 99 packets, got 0 answers, remaining 1 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packet

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


KeyboardInterrupt: 

In [99]:
reply.show()

###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 44
  id        = 0
  flags     = DF
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = 0x5a78
  src       = 192.168.47.129
  dst       = 192.168.47.130
  \options   \
###[ TCP ]### 
     sport     = http
     dport     = ftp_data
     seq       = 3165489273
     ack       = 1
     dataofs   = 6
     reserved  = 0
     flags     = SA
     window    = 5840
     chksum    = 0x5b66
     urgptr    = 0
     options   = [('MSS', 1460)]
###[ Padding ]### 
        load      = '\x00\x00'



## Create ping request
- IP packet with ICMP echo request and some payload
- send the packet and capture it with wireshark

In [120]:
# using spoofed IP
payload = "Tell me if you're up..."
ping = IP(dst=f"{host}") / ICMP() / payload 

In [121]:
ping.show()

###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = icmp
  chksum    = None
  src       = 192.168.47.130
  dst       = 192.168.47.129
  \options   \
###[ ICMP ]### 
     type      = echo-request
     code      = 0
     chksum    = None
     id        = 0x0
     seq       = 0x0
     unused    = ''
###[ Raw ]### 
        load      = "Tell me if you're up..."



In [122]:
send(ping)

.
Sent 1 packets.


In [123]:
# you'll never receive the ping response, if your spoof the source destination
reply = sr1(ping)
reply.show()

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 51
  id        = 25620
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = icmp
  chksum    = 0x3662
  src       = 192.168.47.129
  dst       = 192.168.47.130
  \options   \
###[ ICMP ]### 
     type      = echo-reply
     code      = 0
     chksum    = 0x621a
     id        = 0x0
     seq       = 0x0
     unused    = ''
###[ Raw ]### 
        load      = "Tell me if you're up..."



In [124]:
reply['Raw']

<Raw  load="Tell me if you're up..." |>

## Exercise

- Spoof src IP and ping some IP in the network you own!
- 